In [1]:
# 5.11.1 残差块
# 残差块里首先有2个有相同输出通道数的 3×3 卷积层。每个卷积层后接一个批量归一化层和ReLU激活函数。
# 然后我们将输入跳过这2个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求2个卷积层的输出与输入形状一样，从而可以相加。
# 如果想改变通道数，就需要引入一个额外的 1×1 卷积层来将输入变换成需要的形状后再做相加运算。
# 残差块的实现如下。它可以设定输出通道数、是否使用额外的 1×1 卷积层来修改通道数以及卷积层的步幅。
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

# 本类已保存在d2lzh包中方便以后使用
class Residual(nn.Block):
    def __init__(self,num_channels,use_1x1conv=False,strides=1,**kwargs):
        super(Residual,self).__init__(**kwargs)
        self.conv1=nn.Conv2D(num_channels,kernel_size=3,padding=1,strides=strides)# 两个3x3卷积层
        self.conv2=nn.Conv2D(num_channels,kernel_size=3,padding=1)
        if use_1x1conv:#是否使用1x1卷积层来修改通道数以及卷积层的步幅。
            self.conv3=nn.Conv2D(num_channels,kernel_size=1,strides=strides)
        else:
            self.conv3=None
        self.bn1=nn.BatchNorm()# 两个批量归一化层
        self.bn2=nn.BatchNorm()
            
    def forward(self,X):
        Y=nd.relu(self.bn1(self.conv1(X)))# 加权运算1(卷积和批量归一化)和激活函数
        Y=self.bn2(self.conv2(Y))# 加权运算2(卷积和批量归一化)，得到f(x)-x
        if self.conv3:
            X=self.conv3(X)
        return nd.relu(Y+X)# Y+X即f(x),在应用激活函数

In [2]:
# 输入和输出形状一致的情况。
blk=Residual(3)
blk.initialize()
X=nd.random.uniform(shape=(4,3,6,6))
blk(X).shape

(4, 3, 6, 6)

In [3]:
# 在增加输出通道数的同时减半输出的高和宽。
blk=Residual(6,use_1x1conv=True,strides=2)
blk.initialize()
blk(X).shape

(4, 6, 3, 3)

In [4]:
# 5.11.2 ResNet模型
# ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、步幅为2的 7×7 卷积层后接步幅为2的 3×3 的最大池化层。
# 不同之处在于ResNet每个卷积层后增加的批量归一化层。
net=nn.Sequential()
net.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3),
       nn.BatchNorm(),nn.Activation('relu'),# 批量归一化发生在卷积计算之后、应用激活函数之前。
       nn.MaxPool2D(pool_size=3,strides=2,padding=1)
       )

In [5]:
# ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。
# 第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。
# 之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。
# 下面我们来实现这个模块。注意，这里对第一个模块做了特别处理。
def resnet_block(num_channels,num_residuals,first_block=False):
    blk=nn.Sequential()
    for i in range(num_residuals):# 残差块数
        if i==0 and not first_block:# 除第一个残差模块的第一个残差块将上一个模块的通道数翻倍，并将高和宽减半。
            blk.add(Residual(num_channels,use_1x1conv=True,strides=2))
        else:
            blk.add(Residual(num_channels))
    return blk

In [6]:
# 每个模块使用2个残差块。
net.add(resnet_block(64,2,first_block=True),# 第一个残差模块不用减半
       resnet_block(128,2),
       resnet_block(256,2),
       resnet_block(512,2))

In [7]:
# 最后，与GoogLeNet一样，加入全局平均池化层后接上全连接层输出。
net.add(nn.GlobalAvgPool2D(),nn.Dense(10))

In [8]:
# 输入形状在ResNet不同模块之间的变化。
X=nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv5 output shape:	 (1, 64, 112, 112)
batchnorm4 output shape:	 (1, 64, 112, 112)
relu0 output shape:	 (1, 64, 112, 112)
pool0 output shape:	 (1, 64, 56, 56)
sequential1 output shape:	 (1, 64, 56, 56)
sequential2 output shape:	 (1, 128, 28, 28)
sequential3 output shape:	 (1, 256, 14, 14)
sequential4 output shape:	 (1, 512, 7, 7)
pool1 output shape:	 (1, 512, 1, 1)
dense0 output shape:	 (1, 10)
